In [1]:
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, GPTSimpleVectorIndex, PromptHelper
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LLMPredictor
import torch
from langchain.llms.base import LLM
from transformers import pipeline

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
device = "cuda" if torch.cuda.is_available() else "cpu"

/home/magic/miniconda3/envs/gpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class FlanLLM(LLM):
    model_name = "../../vicuna-7b"
    pipeline = pipeline("text-generation", model=model_name, device=0, model_kwargs={
        "torch_dtype": torch.bfloat16
    })

    def _call(self, prompt, stop=None):
        return self.pipeline(prompt, max_length=9999)[0]["generated_text"]

    def _identifying_params(self):
        return {"name_of_model": self.model_name}

    def _llm_type(self):
        return "custome"


Loading checkpoint shards: 100%|██████████| 2/2 [00:34<00:00, 17.15s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 23.62 GiB total capacity; 22.63 GiB already allocated; 37.56 MiB free; 22.63 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
llm_predictor = LLMPredictor(llm=FlanLLM())
hfemb = HuggingFaceEmbeddings()
embed_model = LangchainEmbedding(hfemb)